# Pandas Groupby compared to SQL

Pandas Dataframes ar very versatile, in terms of their capability to manipulate, reshape and munge data. The most important feature of a DataFrame is its capability to aggregate data. 

Most often, the aggregation capability is compared to the GROUP BY facility in SQL. However, there are fine differences between how SQL GROUP BY and groupby() in Pandas DataFrame operates. 

We will look at following SQL features and equivalent pandas way of achieving the same.
* GROUP BY & Aggregation 
* Count of grouped records
* HAVING clause

We have a pandas DataFrame, with countries, cities, their temperatures and relative humidities. The same data I have it in a MySQL table, named city_weather.

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'country' : ['India','India','India', 'North America', 'North America', 'United Kingdom', 'Russia','Russia', 'Brazil','Brazil', 'Germany', 'Germany','Germany','Poland'],
                   'city' : ['Delhi', 'Bangalore', 'Mumbai', 'New York', 'Chicago', 'London', 'Moscow', 'Samara','Sao Paulo','Rio de Janeiro','Berlin','Hamburg','Munich','Warsaw'],
                   'temperature' : [40,29,33,21,17,12,19,24,14,19,17,16,15,17],
                   'humidity' : [12,63,61,55,69,88,56,41,85,90,62,58,90,66]})
df

,city,country,humidity,temperature
0,Delhi,India,12,40
1,Bangalore,India,63,29
2,Mumbai,India,61,33
3,New York,North America,55,21
4,Chicago,North America,69,17
5,London,United Kingdom,88,12
6,Moscow,Russia,56,19
7,Samara,Russia,41,24
8,Sao Paulo,Brazil,85,14
9,Rio de Janeiro,Brazil,90,19


## Simple Group By and Aggregation

Suppose say, I want to find the lowest temperature for each country:

In SQL, we would write:

In [ ]:
mysql> SELECT country, min(temperature) as mintemp from city_weather GROUP BY country;
+----------------+---------+
| country        | mintemp |
+----------------+---------+
| Brazil         |      14 |
| Germany        |      15 |
| India          |      29 |
| North America  |      17 |
| Poland         |      17 |
| Russia         |      19 |
| United Kingdom |      12 |
+----------------+---------+
7 rows in set (0.00 sec)

The *min()* function is an aggregation and *group by* is the SQL operator for grouping. After forming groups of records for each country, it finds the minimum temperature for each group and prints the grouping keys and the aggregated values.

In SQL, applying *group by* and applying aggregation function on selected columns happen as a single operation. If a group by is applied, then any column in the *select* list must either be part of the *group by* clause or must be aggregated using aggregation functions like count(), sum(), avg() etc.

Let's do the same in Pandas:

In [3]:
grp=df.groupby('country')

grp['temperature'].min()

country
Brazil            14
Germany           15
India             29
North America     17
Poland            17
Russia            19
United Kingdom    12
Name: temperature, dtype: int64

Dataframe.groupby() function returns a **DataFrameGroupBy** object. Here we did the grouping based on country column. 

The groupby operation works a little bit different here compared to SQL. The given key value (in this case, country) is used to split the original dataframe into groups. 

The grp object contains as many groups as there are unique country names. Each group is named after the key value that was used to split the original DataFrame.

In this example, groups will be named after the country names. (see below). The groups object can be visualised as a dictionary, having groupnames as keys. The value corresponding to key is a list of indices pointing to the set of rows that fall under that group.

In [4]:
grp.groups

{'Brazil': Int64Index([8, 9], dtype='int64'),
 'Germany': Int64Index([10, 11, 12], dtype='int64'),
 'India': Int64Index([0, 1, 2], dtype='int64'),
 'North America': Int64Index([3, 4], dtype='int64'),
 'Poland': Int64Index([13], dtype='int64'),
 'Russia': Int64Index([6, 7], dtype='int64'),
 'United Kingdom': Int64Index([5], dtype='int64')}


See the *describe()* output of *grp*. The summary is for each group.

In [5]:
grp.describe()

humidity                                                        \
                  count       mean        std   min    25%   50%    75%   max   
country                                                                         
Brazil              2.0  87.500000   3.535534  85.0  86.25  87.5  88.75  90.0   
Germany             3.0  70.000000  17.435596  58.0  60.00  62.0  76.00  90.0   
India               3.0  45.333333  28.884829  12.0  36.50  61.0  62.00  63.0   
North America       2.0  62.000000   9.899495  55.0  58.50  62.0  65.50  69.0   
Poland              1.0  66.000000        NaN  66.0  66.00  66.0  66.00  66.0   
Russia              2.0  48.500000  10.606602  41.0  44.75  48.5  52.25  56.0   
United Kingdom      1.0  88.000000        NaN  88.0  88.00  88.0  88.00  88.0   

               temperature                                                  
                     count  mean       std   min    25%   50%    75%   max  
country                                                                     
Brazil                 2.0  16.5  3.535534  14.0  15.25  16.5  17.75  19.0  
Germany                3.0  16.0  1.000000  15.0  15.50  16.0  16.50  17.0  
India                  3.0  34.0  5.567764  29.0  31.00  33.0  36.50  40.0  
North America          2.0  19.0  2.828427  17.0  18.00  19.0  20.00  21.0  
Poland                 1.0  17.0       NaN  17.0  17.00  17.0  17.00  17.0  
Russia                 2.0  21.5  3.535534  19.0  20.25  21.5  22.75  24.0  
United Kingdom         1.0  12.0       NaN  12.0  12.00  12.0  12.00  12.0

Now lets print each group

In [6]:
for grpname, grpdata in grp:
    print(grpname)
    print(grpdata)

Brazil
             city country  humidity  temperature
8       Sao Paulo  Brazil        85           14
9  Rio de Janeiro  Brazil        90           19
Germany
       city  country  humidity  temperature
10   Berlin  Germany        62           17
11  Hamburg  Germany        58           16
12   Munich  Germany        90           15
India
        city country  humidity  temperature
0      Delhi   India        12           40
1  Bangalore   India        63           29
2     Mumbai   India        61           33
North America
       city        country  humidity  temperature
3  New York  North America        55           21
4   Chicago  North America        69           17
Poland
      city country  humidity  temperature
13  Warsaw  Poland        66           17
Russia
     city country  humidity  temperature
6  Moscow  Russia        56           19
7  Samara  Russia        41           24
United Kingdom
     city         country  humidity  temperature
5  London  United Kingdom      

If you look closely, after the groupby operation, the keys used for grouping has become the index of the resultant groupby object.

This can be prevented by specifying *as_index=False*. If you see below, the key has been retained as a column and the index is defaulted to integer values.

In [101]:
grp=df.groupby('country', as_index=False)

grp['temperature'].min()

,country,temperature
0,Brazil,14
1,Germany,15
2,India,29
3,North America,17
4,Poland,17
5,Russia,19
6,United Kingdom,12


## Selecting a non-Group By, non-Aggregated column from a grouped object

Now, lets say, we want to find the city having lowest temperature for each country. 

This becomes a bit tricky in SQL, because, when we do a group by, it cannot contain a column that is not used for grouping or aggregation. city column is not used for grouping. Below SQL is not valid:

**mysql> select country, city, min(temperature) from city_weather GROUP BY country;**
ERROR 1055 (42000): Expression #2 of SELECT list is not in GROUP BY clause and contains nonaggregated column 'junkdb.city_weather.city' which is not functionally dependent on columns in GROUP BY clause; this is incompatible with sql_mode=only_full_group_by

So, we will end up joining two SQLs as shown below:

select A.country,A.city,A.temperature 
from city_weather A, 
(select country,min(temperature) as mintemp from city_weather group by country) B 
where A.country = B.country 
and A.temperature = B.mintemp;

Here, we calculate the minimum temperature per country in one query (alias B), and join it with the full table (alias A) on country and minimum temperature. Records whose temperature values matching minimum temperature under the given country group will get returned: 

In [ ]:
mysql> select A.country,A.city,A.humidity,A.temperature from city_weather A, 
(select country,min(temperature) as mintemp from city_weather group by country)B 
where A.country = B.country and A.temperature = B.mintemp;
+----------------+-----------+----------+-------------+
| country        | city      | humidity | temperature |
+----------------+-----------+----------+-------------+
| India          | Bangalore |       63 |          29 |
| North America  | Chicago   |       69 |          17 |
| United Kingdom | London    |       88 |          12 |
| Russia         | Moscow    |       56 |          19 |
| Brazil         | Sao Paulo |       85 |          14 |
| Germany        | Munich    |       90 |          15 |
| Poland         | Warsaw    |       66 |          17 |
+----------------+-----------+----------+-------------+
7 rows in set (0.00 sec)

How do we do it in pandas ?

Well, its not that complicated. Unlike SQL, individual groups in the DataFrameGroupBy object contains the full record. So, we can retrieve any column value from the group. 

The *idxmin()* function of a dataframe, when invoked on a column, returns the index (row label) of the row having minimum value of the column. In this case, we can apply the idxmin() function on the temperature column of the grouped dataframe. 

The index values thus obtained, can be used to retrieve the complete row.

In [119]:
df.groupby('country')['temperature'].idxmin()

country
Brazil             8
Germany           12
India              1
North America      4
Poland            13
Russia             6
United Kingdom     5
Name: temperature, dtype: int64

Now use the .loc[] on the original data frame, with the set of indices obtained from idxmin() function.

In [120]:
df.loc[df.groupby('country')['temperature'].idxmin()]

,city,country,humidity,temperature
8,Sao Paulo,Brazil,85,14
12,Munich,Germany,90,15
1,Bangalore,India,63,29
4,Chicago,North America,69,17
13,Warsaw,Poland,66,17
6,Moscow,Russia,56,19
5,London,United Kingdom,88,12


## Counting records within groups

In [ ]:
How do we count cities under each country ?

In SQL, this is a straight forward query:

In [ ]:
mysql> select country,count(*) from city_weather group by country;                                                     +----------------+----------+
| country        | count(*) |
+----------------+----------+
| Brazil         |        2 |
| Germany        |        3 |
| India          |        3 |
| North America  |        2 |
| Poland         |        1 |
| Russia         |        2 |
| United Kingdom |        1 |
+----------------+----------+
7 rows in set (0.00 sec)

Here the *count()* returns the count of records under each group (country). The argument inside the *count()* function does not have any meaning here. You use a column name or a constant value as an argument.

How do we do it in pandas ? 

In [124]:
df.groupby('country')['city'].count()
#df.groupby('country', as_index=False)['city'].count()

country
Brazil            2
Germany           3
India             3
North America     2
Poland            1
Russia            2
United Kingdom    1
Name: city, dtype: int64

In SQL world, the same query can be used irrespective of the number of columns that you want to use in group by. Just need to add the column to the group by clause as well as the select clause. count(*) function does not require a column to count records.

In pandas, the count() function requires atleast one column that does not take part in the grouping operation, to count. This gets a little tricky, when you want to group by all columns in a dataframe. See the example below:

In [126]:
df = pd.DataFrame({'make':['alfa-romeo','alfa-romeo','alfa-romeo','audi','audi'], 'body-style':['convertible','convertible','hatchback','sedan','sedan']})

df

,body-style,make
0,convertible,alfa-romeo
1,convertible,alfa-romeo
2,hatchback,alfa-romeo
3,sedan,audi
4,sedan,audi


Now, I want to find the counts of each combination. It is a group by on mutiple columns. My output must look like: 

In [ ]:
             make   body-style   count
0     alfa-romero  convertible       2
1     alfa-romero    hatchback       1
2            audi    sedan           2

If we do as shown below, then there are no columns left to count. So there wont be any count atall.

In [128]:
df.groupby(by=['make','body-style']).count()
#df.groupby(by=['make','body-style'], as_index=False).count()

Empty DataFrame
Columns: []
Index: [(alfa-romeo, convertible), (alfa-romeo, hatchback), (audi, sedan)]

In such scenarios, we must use the size() function on each group. 

Caveat:
while .count() is an aggregation function, .size() is not. It will not skip NaN values as .count() does. So we have to be careful while using size().

In [148]:
df.groupby(by=['make','body-style'], as_index=False).size()

make        body-style 
alfa-romeo  convertible    2
            hatchback      1
audi        sedan          2
dtype: int64

If you want to assign a name to the count column, assign it to the name variable and then reset_index

In [150]:
c=df.groupby(by=['make','body-style'], as_index=False).size()
c.name="city_count"
c=c.reset_index()
c

,make,body-style,city_count
0,alfa-romeo,convertible,2
1,alfa-romeo,hatchback,1
2,audi,sedan,2


## SQL HAVING clause and Pandas

In SQL, a HAVING clause is used to filter records after a GROUP BY is applied. 
See the below example:

In [ ]:
mysql> select country, min(temperature) as mintemp from city_weather GROUP BY country HAVING mintemp < 15;
+----------------+---------+
| country        | mintemp |
+----------------+---------+
| Brazil         |      14 |
| United Kingdom |      12 |
+----------------+---------+
2 rows in set (0.00 sec)

When the HAVING clause is applied on a GROUP BY, it discards grouped records that do not satisfy the criteria - mintemp < 15.

Now, Lets see the equivalent of HAVING clause in pandas dataframe.


In [14]:
grp=df.groupby('country', as_index=False)['temperature'].min()
grp

,country,temperature
0,Brazil,14
1,Germany,15
2,India,29
3,North America,17
4,Poland,17
5,Russia,19
6,United Kingdom,12


We prepare DataFrameGroupBy object as we did before. So we got country-wise minimum temperatures. Now, to mimic a HAVING clause, we must do a filter on the grp object. 

In [13]:
grp[grp['temperature'] < 15]

,country,temperature
0,Brazil,14
6,United Kingdom,12


In [ ]:
There we have it. 